# Determination of the precision
Determine the average localization precision per target based on the individual errors per localization in nm. <br/>

In swift, the 'precision' parameter is relevant for detecting immobile particles as their apparent displacement is a result of the localization error.

## Choose a localization software

In [1]:
from pySPT.widgets import widgetDirectoryStructure
from pySPT.widgets import widgetColumnSort
from pySPT.widgets import widgetPrecision
from pySPT.preAnalysis import precision
widget_precision = widgetPrecision.WidgetPrecision(pixel_size="158", camera_dt="0.02")  # adjust the default parameters
widget_dir_structure = widgetDirectoryStructure.WidgetDirStructure()
precision = precision.Precision()
display(widget_precision.software_button)

RadioButtons(options=('ThunderSTORM', 'rapidSTORM'), value='ThunderSTORM')

## Precision per folder

A list of localization uncertainties per folder is saved and should be used to write the swift batch script with individual precision values.

### Define directory
The precision is calculated for all *.csv files in the directory.

In [2]:
display(widget_precision.dir_box, widget_precision.dir_button)
widget_precision.dir_button.on_click(widget_precision.open_dir)
widget_precision.dir_box.observe(widget_precision.change_dir_box)

Text(value='', description='Directory', placeholder='directory to be searched in', style=DescriptionStyle(desc…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

### Run

In [3]:
display(widget_precision.run_button_folder)
def run_analysis(event):
    widget_precision.create_clear_output()
    display(widget_precision.run_button_folder)
    files, file_names = precision.get_loc_files(widget_precision.dir_box.value)
    precision.get_precisions(files, file_names)
    precision.plot_box()
    precision.analysis_executed = True
widget_precision.run_button_folder.on_click(run_analysis)

Button(description='run', style=ButtonStyle(), tooltip='run the analysis')

### Save
Define a directory and folder name to save the results. Save the plots optionally.

In [4]:
display(widget_precision.box_foldername, widget_precision.dir_box_save, widget_precision.dir_button_save)
widget_precision.dir_button_save.on_click(widget_precision.open_dir_save)
widget_precision.dir_box_save.observe(widget_precision.change_dir_box_save)

Text(value='precision', description='Foldername', placeholder='name of folder', style=DescriptionStyle(descrip…

Text(value='', description='Directory', placeholder='Directory to save', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [5]:
display(widget_precision.save_fig_checkbox, widget_precision.save_button_folder)
def save_analysis(event):
    widget_precision.create_clear_output()
    display(widget_precision.save_fig_checkbox, widget_precision.save_button_folder)
    if precision.analysis_executed:
        _, file_names = precision.get_loc_files(widget_precision.dir_box.value)
        precision.save_precision_list(widget_precision.dir_box_save.value + "\\" + widget_precision.box_foldername.value, precision.mean_values, widget_precision.save_fig_checkbox.value, file_names)
    else:
        print("Please run the analysis first, by clicking at the 'run' button.")
widget_precision.save_button_folder.on_click(save_analysis)

Checkbox(value=True, description='Save plot')

Button(description='save', style=ButtonStyle(), tooltip='save the results')

## Precision per file

The execution per file is recommended if a visualization of the localization uncertainty distribution should be created or if hidden markov modeling is using the static error to correct the diffusion coefficients.

### Load file

thunderSTORM: Insert the full file path of the thunderSTORM exported results table with .csv ending or browse for it. Make sure that the uncertainty is in nm. (Example path: F:\data_folder\cell01.csv) <br/>
rapidSTORM: Insert the full file path of the rapidSTORM localization file with .txt ending or browse for it. (Example path: F:\data_folder\cell01.txt) 

In [6]:
display(widget_precision.file_text_box, widget_precision.file_button)
widget_precision.file_button.on_click(widget_precision.open_file)
widget_precision.file_text_box.observe(widget_precision.change_file_box)

Text(value='', description='Complete path', placeholder='insert path', style=DescriptionStyle(description_widt…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Run analysis

The uncertainties are ln-transformed and fitted with a Gauss-fit (to visually check for a normal distribution). The retransformed mean value of the ln(uncertainties) represents the mean position uncertainty in the x/y-plane.

In [7]:
display(widget_precision.run_button)
def run_analysis(event):
    widget_precision.create_clear_output()
    display(widget_precision.run_button)
    if widget_precision.got_file_name:
        if widget_precision.software_button.value == "ThunderSTORM":
            widget_column_sort = widgetColumnSort.WidgetColumnSort(widget_precision.file_text_box.value, "ThunderSTORM", [('"uncertainty_xy [nm]"', '"uncertainty [nm]"')])
            precision.software = widget_precision.software_button.value
        elif widget_precision.software_button.value == "rapidSTORM":
            widget_column_sort = widgetColumnSort.WidgetColumnSort(widget_precision.file_text_box.value, "rapidSTORM", [('"Position-0-0-uncertainty"',), ('"Position-1-0-uncertainty"',)])
            precision.software = widget_precision.software_button.value
        widget_column_sort.check_header()
        if widget_column_sort.correct_header:
            widget_column_sort.run_column_sort()
            precision.file_name = widget_precision.file_text_box.value
            precision.column_order = widget_column_sort.column_order
            precision.run_precision()
        else:
            widget_precision.warning_wrong_file()
    else:
        widget_precision.warning_wrong_file_path()
widget_precision.run_button.on_click(run_analysis)

Button(description='run', style=ButtonStyle(), tooltip='run the analysis')

### Save

The results will be saved as .txt files (one for each histogram and one for the localization precision) in the preAnalysis folder of the target file. Save the plots optionally.

In [8]:
display(widget_precision.save_figures_checkbox, widget_precision.save_button)
def save_analysis(event):
    widget_precision.create_clear_output()
    display(widget_precision.save_figures_checkbox, widget_precision.save_button)
    widget_dir_structure.name_handling(widget_precision.file_text_box.value)
    widget_dir_structure.create_raw_base_name()
    widget_dir_structure.sub_folder = "\\preAnalysis"
    widget_dir_structure.create_folder()
    precision.save_precision(widget_dir_structure.sub_folder_dir, widget_dir_structure.raw_base_name)
    if widget_precision.save_figures_checkbox.value:
        precision.run_save_plots(widget_dir_structure.sub_folder_dir, widget_dir_structure.raw_base_name)
widget_precision.save_button.on_click(save_analysis)

Checkbox(value=True, description='Save plots')

Button(description='save', style=ButtonStyle(), tooltip='save the results')